In [190]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [191]:
train = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')

all_data = pd.concat([train,test])

%matplotlib inline
all_data.columns

In [192]:
all_data.head(10)

In [193]:
all_data.info()

In [194]:
#A better way to understand numeric data is to use describe()
train.describe()

In [195]:
train_num = train[['Age', 'SibSp', 'Parch', 'Fare']]
train_cat = train[[ 'Sex', 'Embarked']]

In [196]:
import matplotlib.pyplot as plt

#numerical distributions
train_num.hist(figsize=(20,15))

Apparently, Age is normally distributed but other columns are not.

In [197]:
train_num.corr()

In [198]:
pd.pivot_table(train, index='Survived', values=['Age','SibSp', 'Parch','Fare'])

In [199]:
for i in train_cat:
    print(train_cat[i].value_counts())

In [200]:
#Dealing with cat data
for col in train_num:
    train[col].fillna(train[col].mean(), inplace=True)
    test[col].fillna(test[col].mean(), inplace=True)

In [201]:
train['Embarked'].fillna('U', inplace=True)
test['Embarked'].fillna('U', inplace=True)

In [202]:
train.head(10)

In [203]:
from sklearn import preprocessing 
encoder = preprocessing.LabelEncoder()
for col in train_cat:
    train[col] = encoder.fit_transform(train[col])
    test[col] = encoder.transform(test[col])
    print(encoder.classes_)

In [204]:
train.head(10)

In [205]:
train = train.drop(['PassengerId','Name','Ticket','Cabin'],axis=1)

In [206]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

y = train['Survived']
X = train.drop('Survived', axis=1)

X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.2, random_state=42)
X_train.head(5)

In [207]:
clf = LogisticRegression(random_state=42, max_iter=1000)
clf.fit(X_train, y_train)

In [208]:
pred = clf.predict(X_val)

In [209]:
from sklearn.metrics import accuracy_score
accuracy_score(pred,y_val)

In [210]:
test_ids = test['PassengerId']

In [211]:
test = test.drop(['PassengerId','Name','Ticket','Cabin'],axis=1)

In [212]:
test.head(5)

In [213]:
submission = clf.predict(test)

In [218]:
submission[:10]

In [232]:
df = pd.DataFrame({'PassengerId':test_ids.values, 'Survived': submission,})

In [233]:
df.to_csv('submission.csv', index=False)